In [1]:
import geopandas as gpd
import fiona
import numpy as np
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import rasterio
import pandas as pd

In [2]:
# data\zoning.gdb
path = "../data/zoning.gdb/"
layers = fiona.listlayers(path)

In [3]:
zoning = gpd.read_file(path, layer=layers[0])

In [4]:
zoning.head()

,ZONEDIST,Shape_Length,Shape_Area,geometry
0,R4-1,2575.578630,372245.385615,"MULTIPOLYGON (((982158.973 167972.613, 981835...."
1,PARK,569.651123,16069.895162,"MULTIPOLYGON (((996439.256 197691.699, 996350...."
2,PARK,719.458367,20178.003443,"MULTIPOLYGON (((988123 179762.232, 988060.069 ..."
3,C4-4D,1147.584820,75618.341291,"MULTIPOLYGON (((1013228.549 184395.28, 1013238..."
4,PARK,465.328947,13176.323383,"MULTIPOLYGON (((1003403.83 186973.91, 1003410...."


In [5]:
minx, miny, maxx, maxy = zoning.total_bounds

grid_size = 3280.84 

grid_width = grid_size
grid_height = grid_size

polygons = []
for i in range(int((maxy - miny) / grid_size)):
    for j in range(int((maxx - minx) / grid_size)):
        x1 = minx + j * grid_width
        y1 = miny + i * grid_height
        x2 = x1 + grid_width
        y2 = y1 + grid_height
        
        polygons.append(Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]))

In [6]:
gdf = gpd.GeoDataFrame({'geometry': polygons})
gdf = gdf.set_crs(zoning.crs, allow_override=True)
gdf.head()

,geometry
0,"POLYGON ((913013.301 118296.425, 916294.141 11..."
1,"POLYGON ((916294.141 118296.425, 919574.981 11..."
2,"POLYGON ((919574.981 118296.425, 922855.821 11..."
3,"POLYGON ((922855.821 118296.425, 926136.661 11..."
4,"POLYGON ((926136.661 118296.425, 929417.501 11..."


In [7]:
joined = gpd.sjoin(gdf, zoning[['geometry', 'ZONEDIST']], how='left', predicate='intersects')
zonedist_per_grid = joined.groupby(joined.index)['ZONEDIST'].unique().reset_index()
gdf = gdf.merge(zonedist_per_grid, left_index=True, right_on='index', how='left')
gdf = gdf.drop(columns=['index'])


In [8]:
gdf.head()

,geometry,ZONEDIST
0,"POLYGON ((913013.301 118296.425, 916294.141 11...","[R3A, PARK]"
1,"POLYGON ((916294.141 118296.425, 919574.981 11...","[R3A, R3-2, R3X, R1-2, PARK]"
2,"POLYGON ((919574.981 118296.425, 922855.821 11...","[R1-2, R3X, R1-1]"
3,"POLYGON ((922855.821 118296.425, 926136.661 11...",[nan]
4,"POLYGON ((926136.661 118296.425, 929417.501 11...",[nan]


In [9]:
datetimes = pd.date_range(start='2024-01-01 00:00', end='2024-06-30 23:00', freq='H')
gdf_expanded = gdf.loc[gdf.index.repeat(len(datetimes))].reset_index(drop=True)
datetime_expanded = pd.concat([pd.DataFrame({'datetime': datetimes})] * len(gdf), ignore_index=True)
gdf_expanded['date'] = datetime_expanded['datetime'].dt.strftime('%Y-%m-%d')
gdf_expanded['time'] = datetime_expanded['datetime'].dt.strftime('%H:%M')

C:\Users\SIZHE\AppData\Local\Temp\ipykernel_23948\1826111297.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  datetimes = pd.date_range(start='2024-01-01 00:00', end='2024-06-30 23:00', freq='H')


In [10]:
gdf_expanded

,geometry,ZONEDIST,date,time
0,"POLYGON ((913013.301 118296.425, 916294.141 11...","[R3A, PARK]",2024-01-01,00:00
1,"POLYGON ((913013.301 118296.425, 916294.141 11...","[R3A, PARK]",2024-01-01,01:00
2,"POLYGON ((913013.301 118296.425, 916294.141 11...","[R3A, PARK]",2024-01-01,02:00
3,"POLYGON ((913013.301 118296.425, 916294.141 11...","[R3A, PARK]",2024-01-01,03:00
4,"POLYGON ((913013.301 118296.425, 916294.141 11...","[R3A, PARK]",2024-01-01,04:00
...,...,...,...,...
9648907,"POLYGON ((1063931.941 269215.065, 1067212.781 ...",[nan],2024-06-30,19:00
9648908,"POLYGON ((1063931.941 269215.065, 1067212.781 ...",[nan],2024-06-30,20:00
9648909,"POLYGON ((1063931.941 269215.065, 1067212.781 ...",[nan],2024-06-30,21:00
9648910,"POLYGON ((1063931.941 269215.065, 1067212.781 ...",[nan],2024-06-30,22:00


In [12]:
result = gdf_expanded[['ZONEDIST', 'geometry', 'date', 'time']]
result

,ZONEDIST,geometry,date,time
0,"[R3A, PARK]","POLYGON ((913013.301 118296.425, 916294.141 11...",2024-01-01,00:00
1,"[R3A, PARK]","POLYGON ((913013.301 118296.425, 916294.141 11...",2024-01-01,01:00
2,"[R3A, PARK]","POLYGON ((913013.301 118296.425, 916294.141 11...",2024-01-01,02:00
3,"[R3A, PARK]","POLYGON ((913013.301 118296.425, 916294.141 11...",2024-01-01,03:00
4,"[R3A, PARK]","POLYGON ((913013.301 118296.425, 916294.141 11...",2024-01-01,04:00
...,...,...,...,...
9648907,[nan],"POLYGON ((1063931.941 269215.065, 1067212.781 ...",2024-06-30,19:00
9648908,[nan],"POLYGON ((1063931.941 269215.065, 1067212.781 ...",2024-06-30,20:00
9648909,[nan],"POLYGON ((1063931.941 269215.065, 1067212.781 ...",2024-06-30,21:00
9648910,[nan],"POLYGON ((1063931.941 269215.065, 1067212.781 ...",2024-06-30,22:00


In [13]:
# save result
result.to_file("../data_nyc/zone.geojson", driver='GeoJSON')
# save result as csv
result.to_csv("../data_nyc/zone.csv", index=False)